In [ ]:
import pandas as pd
import numpy as np

In [ ]:
titanic_df =pd.read_csv(r'F:\SHIKHER-VS\DSMP\02_pandas\datasets\dataset_extra\titanic.csv')
titanic_df.head(4)

In [ ]:
pclass_groups = titanic_df.groupby('Pclass')

pclass_groups.groups.keys()

In [ ]:
for group in list(pclass_groups.groups.keys()):
    print(f"Pclass: {group}\nAverage Age: {pclass_groups.get_group(group)['Age'].mean()}\nTotal Missing Value: {pclass_groups.get_group(group)['Age'].isna().sum()}\n")

In [ ]:

titanic_df['Embarked'].fillna(titanic_df['Embarked'].mode()[0]).value_counts(ascending = False)


In [ ]:
for group in list(pclass_groups.groups.keys()):
    mode=pclass_groups.get_group(group)['Embarked'].mode()
    pclass_groups.get_group(group)['Embarked'].fillna(mode)
    print(pclass_groups.get_group(group)['Embarked'].value_counts(ascending = False))
    


In [ ]:
my_dict = {}

embarked_grps = titanic_df.groupby("Embarked")
for embarked_grp in list(embarked_grps.groups.keys()):
    pclass_grps = embarked_grps.get_group(embarked_grp).groupby('Pclass')
    my_dict[embarked_grp] = {}
    # print(pclass_grps)
    for pclass_grp in list(pclass_grps.groups.keys()):
        
        # print(round(pclass_grps.get_group(pclass_grp)['Fare'].mean(),2))
        # print(round(pclass_grps.get_group(pclass_grp)['Fare'].agg(['mean','min','max']),2))
        my_dict[embarked_grp][pclass_grp] = round(pclass_grps.get_group(pclass_grp)['Fare'].mean(),2)
my_dict

In [ ]:
fifa_df = pd.read_csv(r"F:\SHIKHER-VS\DSMP\02_pandas\datasets\dataset_extra\Fifa Worldcup 2022 - Sheet1.csv")
fifa_df.head(3)

In [ ]:
def z_normalization(group,cols_to_perform):
    for col in cols_to_perform:
        mean = group[col].mean()
        std = group[col].std()
        group[f"{col}_z_norm"] = (group[col] - mean) / std
        
    return group


In [ ]:
cols_to_perform = ["Passes","Passes Completed","Attempted Line Breaks","Completed Line Breaks"]

groups = fifa_df.groupby("Team")
groups = groups.apply(z_normalization,cols_to_perform = cols_to_perform)
# groups=groups.groupby("Team")
groups.agg(
    {
        "Passes":['min','max'],
        "Yellow Cards":['min','max','mean'],
        "Attempted Line Breaks":['min','max','std'],
        "Possession (%)":'mean'
    }
)

In [ ]:
balls = pd.read_csv(r'F:\SHIKHER-VS\DSMP\02_pandas\datasets\dataset_extra\deliveries.csv')

balls.head(14)

In [ ]:
df = balls[balls.inning == 2]
df.groupby(['match_id', 'batsman']).sum().sort_values(by ="batsman_runs",ascending = False)['batsman_runs'].reset_index()

In [ ]:
t_df = df.groupby('batsman').agg({
    "batsman_runs":"sum",
    'ball':"count"
    
})

t_df["strike_rate"] = t_df["batsman_runs"] / t_df['ball']  * 100

t_df[t_df['ball'] >= 100].sort_values(by ="batsman_runs",ascending =False)

In [ ]:
balls["IsBatsmanOut"] = balls['player_dismissed']==balls['batsman']
balls.groupby(['bowler','batsman']).agg({
    "IsBatsmanOut":'sum',
    "batsman_runs":"sum"
}).sort_values(by= ['IsBatsmanOut','batsman_runs'], ascending =[0,1]).reset_index()
balls

In [ ]:
def fx(x):
    return '-'.join(list(np.sort(x.values)))

balls['Batter-pair'] = balls[['batsman','non_striker']].apply(fx,axis = 1)
balls.groupby('Batter-pair')["total_runs"].sum().sort_values(ascending = 0)

In [84]:
tdf= balls.groupby('Batter-pair').agg({
    'total_runs':'sum',
    'ball':"count",
    'IsBatsmanOut':"sum"

}).reset_index()

tdf["batsman 1"] = tdf["Batter-pair"].apply(lambda x: x.split("-")[0])
tdf["batsman 2"] = tdf["Batter-pair"].apply(lambda x: x.split("-")[1])
tdf.rename(columns = {'total_runs':'Runs'}, inplace  = True)
tdf['Strike Rate'] = tdf['Runs'] / tdf['ball'] * 100
tdf['Average'] = tdf['Runs'] / tdf['IsBatsmanOut'] 

tdf.sort_values('Strike Rate',ascending = 0,inplace = True)

tdf[["batsman 1","batsman 2",'Runs','Average','Strike Rate']]

,batsman 1,batsman 2,Runs,Average,Strike Rate
2071,JP Duminy,NM Coulter,6,inf,600.000000
2061,JP Duminy,KH Pandya,6,inf,600.000000
1152,CL White,P Kumar,12,inf,600.000000
1904,J Bairstow,YK Pathan,6,inf,600.000000
3232,S Aravind,TM Head,16,inf,533.333333
...,...,...,...,...,...
543,AM Rahane,SP Goswami,0,0.0,0.000000
2600,MA Starc,S Aravind,0,NaN,0.000000
2574,M Vohra,P Kumar,0,0.0,0.000000
3278,S Sreesanth,SJ Srivastava,0,0.0,0.000000
